# Historical climate for all glaciers (in OGGM format)

In [ ]:
# basic
from tqdm.notebook import tqdm
import pandas as pd
import os

# spatial
import geopandas as gpd
import xarray as xr
import xesmf as xe

# xclim
from xclim import core

In [ ]:
os.chdir('/home/rooda/OneDrive/Patagonia/')
period = slice("1980-01-01", "2020-12-31")

In [ ]:
# template and source files file
example    = xr.open_dataset("/home/rooda/OneDrive/Coding/OGGM/example_nc.nc")
dem        = xr.open_dataset("GIS South/dem_patagonia005.tif").rename({'y':'latitude', 'x':'longitude', 'band_data':'hgt'})
dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)

# PMET v1.1
pp_pmet    = xr.open_dataset("Data/Zenodo/v11/PP_PMETsim_1980_2020_v11d.nc",   chunks = chunks_dict).sel(time = period)
t2m_pmet   = xr.open_dataset("Data/Zenodo/v11/Tavg_PMETsim_1980_2020_v11d.nc", chunks = chunks_dict).sel(time = period)

In [ ]:
# actual prepro
regridder  = xe.Regridder(dem, pp_pmet, "nearest_s2d")
dem        = regridder(dem)
t2m_pmet   = regridder(t2m_pmet)

pmet   = xr.merge([pp_pmet,  t2m_pmet, dem]).rename({'pp':'prcp', 't2m':'temp'})

pmet.prcp.attrs['units']  = "mm"
pmet.temp.attrs['units']  = "degc"
pmet.hgt.attrs['units']   = "m"

pmet   = pmet.where(mask_pmet, drop = True)

pmet.to_netcdf("/home/rooda/Hydro_results/PMETsim_historical_OGGM.nc")